In [1]:
import tensorflow as tf

2024-04-01 15:13:36.659132: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-01 15:13:36.922661: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-01 15:13:36.922844: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-01 15:13:36.962959: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-01 15:13:37.053708: I tensorflow/core/platform/cpu_feature_guar

In [24]:
from tensorflow.keras.layers import Conv1D, Dense, Activation, Multiply, Input, LSTM, TimeDistributed, Flatten, Reshape
from tensorflow.keras.layers import AvgPool1D, Dropout, Concatenate
from tensorflow.keras.models import Model



# Create a wave block
def wave_block(input_layer, filters, kernel_size, dilation_rate):
    # Create a convolution layer
    conv = Conv1D(filters, kernel_size, padding='causal', dilation_rate=dilation_rate)(input_layer)
    
    # Create a gated activation unit
    tanh = Activation('tanh')(conv)
    sigmoid = Activation('sigmoid')(conv)
    gated = Multiply()([tanh, sigmoid])
    
    output = Conv1D(1, 1)(gated)
        
    return output

# Create a layer to flip
class flip(tf.keras.layers.Layer):
    def __init__(self):
        super(flip, self).__init__()
        
    def call(self, input):
        return tf.reverse(input, axis=[1])

# Create a wavenet model
def wavenet_LSTM(input_shape): 
    # Define the input layer
    input_layer = Input(shape=input_shape)
    
    # First Path
    wave_block1 = wave_block(input_layer, 16, 3, 8)
    wave_block1 = AvgPool1D(pool_size=(10))(wave_block1)
    
    wave_block2 = wave_block(wave_block1, 32, 3, 5)
    wave_block2 = AvgPool1D(pool_size=(10))(wave_block2)

    wave_block3 = wave_block(wave_block2, 64, 3, 3)
    wave_block3 = AvgPool1D(pool_size=(10))(wave_block3)

    wave_block4 = wave_block(wave_block3, 64, 2, 2)
    wave_block4 = AvgPool1D(pool_size=(2))(wave_block4)

    lstm = LSTM(64)(wave_block4)
    dropout = Dropout(0.5)(lstm)

    # Second Path
    flipped = flip()(input_layer)
    flipped = Reshape((30 ,500, 20))(flipped)
    timelstm = TimeDistributed(LSTM(64))(flipped)
    attention = TimeDistributed(Dense(1))(timelstm)
    lstm2 = LSTM(64, return_sequences=True)(attention)
    dropout2 = Dropout(0.2)(lstm2)
    dense = Dense(2)(dropout2)
    tanh = Activation('tanh')(dense)
    flatten = Flatten()(tanh)

    # Concatenate outputs of path1 and path2
    concat = Concatenate()([dropout, flatten])

    # Apply softmax
    output_layer = Dense(2, activation='softmax')(concat)
    
    # Create the model
    model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
    
    return model

In [25]:
model = wavenet_LSTM((15000, 20))

In [26]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_11 (InputLayer)       [(None, 15000, 20)]          0         []                            
                                                                                                  
 conv1d_68 (Conv1D)          (None, 15000, 16)            976       ['input_11[0][0]']            
                                                                                                  
 activation_71 (Activation)  (None, 15000, 16)            0         ['conv1d_68[0][0]']           
                                                                                                  
 activation_72 (Activation)  (None, 15000, 16)            0         ['conv1d_68[0][0]']           
                                                                                            